In [1]:
import pandas as pd
import numpy as np

# 可視化用のライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import neologdn
import MeCab

import re

from tqdm import tqdm_notebook as tqdm
from tqdm._tqdm_notebook import tqdm_notebook
# # プログレスバーのラベル設定
tqdm_notebook.pandas(desc="progress: ")

In [2]:
# 予めtf値を紐付けていたcsvファイルを読み込む
# shift-jisのcsvファイル読み込み
contents_merge_df = pd.read_csv('contents_merge_df.csv', encoding='shift-jis')
contents_merge_df.head()

内容       original_type  \
0  neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             (_記号-一般   
1  neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             )_記号-一般   
2  neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             ._記号-一般   
3  neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...              1_名詞-数   
4  neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...  NEologd_名詞-固有名詞-一般   

   count  word_count        tf  
0      1          26  0.038462  
1      1          26  0.038462  
2      1          26  0.038462  
3      1          26  0.038462  
4      1          26  0.038462

## IDF値算出

In [3]:
# idfの分子（全文書数）
len(contents_merge_df['内容'].value_counts())

4

In [4]:
# idfの分母「各単語が出現する文書の数」の算出
# [original_type]×[内容]のクロス集計表を作成

idf_pivot_df = pd.pivot_table(contents_merge_df, index='original_type', columns='内容', aggfunc=len, values='count')
#tf_pivot_df = pd.pivot_table(contents_merge_df, index='original_type', columns='内容', values='count')#aggfuncを指定しないと[count]カラムの数字が入る
idf_pivot_df

内容                  neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表示される  \
original_type                                                             
')_記号-一般                                                          NaN     
('＼_記号-一般                                                         NaN     
()_名詞-固有名詞-一般                                                     NaN     
(_記号-一般                                                           1.0     
)_記号-一般                                                           1.0     
._記号-一般                                                           1.0     
1_名詞-数                                                            1.0     
1つ_名詞-一般                                                          NaN     
NEologd_名詞-固有名詞-一般                                                1.0     
Parse_名詞-固有名詞-一般                                                  1.0     
__記号-一般                                                           1.0     
n_名詞-固有名詞-組織                                                      NaN     
split_名詞-一般                                                       NaN     
split_名詞-固有名詞-組織                                                  NaN     
t_名詞-固有名詞-組織                                                      NaN     
tagger_名詞-固有名詞-一般                                                 1.0     
text_名詞-固有名詞-一般                                                   1.0     
wakatid_名詞-一般                                                     NaN     
、_記号-読点                                                           1.0     
。_記号-句点                                                           NaN     
「_記号-括弧開                                                          NaN     
「＼_記号-一般                                                          NaN     
」_記号-括弧閉                                                          NaN     
いる_動詞-非自立                                                         NaN     
が_助詞-接続助詞                                                         NaN     
が_助詞-格助詞-一般                                                       1.0     
する_動詞-自立                                                          1.0     
た_助動詞                                                             NaN     
たい_助動詞                                                            NaN     
ため_名詞-非自立-副詞可能                                                    NaN     
...                                                               ...     
に_助詞-格助詞-一般                                                       NaN     
の_助詞-連体化                                                          1.0     
ので_助詞-接続助詞                                                        NaN     
は_助詞-係助詞                                                          NaN     
まずは_接続詞                                                           NaN     
よって_接続詞                                                           NaN     
れる_動詞-接尾                                                          1.0     
を_助詞-格助詞-一般                                                       NaN     
リスト_名詞-一般                                                         NaN     
使用_名詞-サ変接続                                                        NaN     
分かち書き_名詞-一般                                                       NaN     
区切り_名詞-一般                                                         NaN     
区切る_動詞-自立                                                         NaN     
単語_名詞-一般                                                          1.0     
原形_名詞-一般                                                          1.0     
各_接頭詞-名詞接続                                                        1.0     
品詞_名詞-一般                                                          1.0     
型_名詞-接尾-一般                                                        NaN     
文字_名詞-一般                                                          NaN     
文字列_名詞-固有名詞-一般                                                    NaN     
次に_接続詞                   

In [5]:
idf_pivot_df1 = idf_pivot_df > 0
idf_pivot_df1# 1以上ならTrue、1未満ならFalse

内容                  neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表示される  \
original_type                                                             
')_記号-一般                                                        False     
('＼_記号-一般                                                       False     
()_名詞-固有名詞-一般                                                   False     
(_記号-一般                                                          True     
)_記号-一般                                                          True     
._記号-一般                                                          True     
1_名詞-数                                                           True     
1つ_名詞-一般                                                        False     
NEologd_名詞-固有名詞-一般                                               True     
Parse_名詞-固有名詞-一般                                                 True     
__記号-一般                                                          True     
n_名詞-固有名詞-組織                                                    False     
split_名詞-一般                                                     False     
split_名詞-固有名詞-組織                                                False     
t_名詞-固有名詞-組織                                                    False     
tagger_名詞-固有名詞-一般                                                True     
text_名詞-固有名詞-一般                                                  True     
wakatid_名詞-一般                                                   False     
、_記号-読点                                                          True     
。_記号-句点                                                         False     
「_記号-括弧開                                                        False     
「＼_記号-一般                                                        False     
」_記号-括弧閉                                                        False     
いる_動詞-非自立                                                       False     
が_助詞-接続助詞                                                       False     
が_助詞-格助詞-一般                                                      True     
する_動詞-自立                                                         True     
た_助動詞                                                           False     
たい_助動詞                                                          False     
ため_名詞-非自立-副詞可能                                                  False     
...                                                               ...     
に_助詞-格助詞-一般                                                     False     
の_助詞-連体化                                                         True     
ので_助詞-接続助詞                                                      False     
は_助詞-係助詞                                                        False     
まずは_接続詞                                                         False     
よって_接続詞                                                         False     
れる_動詞-接尾                                                         True     
を_助詞-格助詞-一般                                                     False     
リスト_名詞-一般                                                       False     
使用_名詞-サ変接続                                                      False     
分かち書き_名詞-一般                                                     False     
区切り_名詞-一般                                                       False     
区切る_動詞-自立                                                       False     
単語_名詞-一般                                                         True     
原形_名詞-一般                                                         True     
各_接頭詞-名詞接続                                                       True     
品詞_名詞-一般                                                         True     
型_名詞-接尾-一般                                                      False     
文字_名詞-一般                                                        False     
文字列_名詞-固有名詞-一般                                                  False     
次に_接続詞                   

In [6]:
# True＝１なのでTureの数を行毎に合計
# それを格納するカラム[word_include_text_count]を追加
# この値がidfの分母となる
idf_pivot_df1['word_include_text_count'] = idf_pivot_df1.sum(axis=1)
idf_pivot_df1

内容                  neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表示される  \
original_type                                                             
')_記号-一般                                                        False     
('＼_記号-一般                                                       False     
()_名詞-固有名詞-一般                                                   False     
(_記号-一般                                                          True     
)_記号-一般                                                          True     
._記号-一般                                                          True     
1_名詞-数                                                           True     
1つ_名詞-一般                                                        False     
NEologd_名詞-固有名詞-一般                                               True     
Parse_名詞-固有名詞-一般                                                 True     
__記号-一般                                                          True     
n_名詞-固有名詞-組織                                                    False     
split_名詞-一般                                                     False     
split_名詞-固有名詞-組織                                                False     
t_名詞-固有名詞-組織                                                    False     
tagger_名詞-固有名詞-一般                                                True     
text_名詞-固有名詞-一般                                                  True     
wakatid_名詞-一般                                                   False     
、_記号-読点                                                          True     
。_記号-句点                                                         False     
「_記号-括弧開                                                        False     
「＼_記号-一般                                                        False     
」_記号-括弧閉                                                        False     
いる_動詞-非自立                                                       False     
が_助詞-接続助詞                                                       False     
が_助詞-格助詞-一般                                                      True     
する_動詞-自立                                                         True     
た_助動詞                                                           False     
たい_助動詞                                                          False     
ため_名詞-非自立-副詞可能                                                  False     
...                                                               ...     
に_助詞-格助詞-一般                                                     False     
の_助詞-連体化                                                         True     
ので_助詞-接続助詞                                                      False     
は_助詞-係助詞                                                        False     
まずは_接続詞                                                         False     
よって_接続詞                                                         False     
れる_動詞-接尾                                                         True     
を_助詞-格助詞-一般                                                     False     
リスト_名詞-一般                                                       False     
使用_名詞-サ変接続                                                      False     
分かち書き_名詞-一般                                                     False     
区切り_名詞-一般                                                       False     
区切る_動詞-自立                                                       False     
単語_名詞-一般                                                         True     
原形_名詞-一般                                                         True     
各_接頭詞-名詞接続                                                       True     
品詞_名詞-一般                                                         True     
型_名詞-接尾-一般                                                      False     
文字_名詞-一般                                                        False     
文字列_名詞-固有名詞-一般                                                  False     
次に_接続詞                   

In [7]:
# 必要なカラム(新規作成カラム、マージするときのキーのカラム）のみ抽出
idf_pivot_df2 = idf_pivot_df1.reset_index()[['original_type', 'word_include_text_count']]
idf_pivot_df2

内容       original_type  word_include_text_count
0             ')_記号-一般                        1
1            ('＼_記号-一般                        1
2        ()_名詞-固有名詞-一般                        1
3              (_記号-一般                        1
4              )_記号-一般                        1
5              ._記号-一般                        1
6               1_名詞-数                        1
7             1つ_名詞-一般                        1
8   NEologd_名詞-固有名詞-一般                        1
9     Parse_名詞-固有名詞-一般                        2
10             __記号-一般                        2
11        n_名詞-固有名詞-組織                        2
12         split_名詞-一般                        1
13    split_名詞-固有名詞-組織                        1
14        t_名詞-固有名詞-組織                        2
15   tagger_名詞-固有名詞-一般                        1
16     text_名詞-固有名詞-一般                        2
17       wakatid_名詞-一般                        1
18             、_記号-読点                        4
19             。_記号-句点                        1
20            「_記号-括弧開                        2
21            「＼_記号-一般                        1
22            」_記号-括弧閉                        3
23           いる_動詞-非自立                        1
24           が_助詞-接続助詞                        1
25         が_助詞-格助詞-一般                        2
26            する_動詞-自立                        4
27               た_助動詞                        2
28              たい_助動詞                        1
29      ため_名詞-非自立-副詞可能                        1
..                 ...                      ...
38         に_助詞-格助詞-一般                        2
39            の_助詞-連体化                        4
40          ので_助詞-接続助詞                        1
41            は_助詞-係助詞                        3
42             まずは_接続詞                        1
43             よって_接続詞                        1
44            れる_動詞-接尾                        2
45         を_助詞-格助詞-一般                        1
46           リスト_名詞-一般                        2
47          使用_名詞-サ変接続                        1
48         分かち書き_名詞-一般                        2
49           区切り_名詞-一般                        1
50           区切る_動詞-自立                        1
51            単語_名詞-一般                        3
52            原形_名詞-一般                        2
53          各_接頭詞-名詞接続                        1
54            品詞_名詞-一般                        2
55          型_名詞-接尾-一般                        2
56            文字_名詞-一般                        1
57      文字列_名詞-固有名詞-一般                        2
58              次に_接続詞                        1
59            省く_動詞-自立                        1
60          群_名詞-接尾-一般                        1
61         行_名詞-接尾-助数詞                        1
62          表示_名詞-サ変接続                        2
63          連続_名詞-サ変接続                        1
64        邪魔_名詞-形容動詞語幹                        1
65             間_名詞-一般                        1
66            関数_名詞-一般                        1
67             ＼_記号-一般                        2

[68 rows x 2 columns]

In [8]:
# 新規に作成した「各単語が出現する文書の数」カラムを[original_type]をキーにして連結
idf_merge_df = pd.merge(contents_merge_df, idf_pivot_df2, on='original_type', how='inner')
idf_merge_df

内容       original_type  \
0    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             (_記号-一般   
1    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             )_記号-一般   
2    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             ._記号-一般   
3    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...              1_名詞-数   
4    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...  NEologd_名詞-固有名詞-一般   
5    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...    Parse_名詞-固有名詞-一般   
6    まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...    Parse_名詞-固有名詞-一般   
7    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             __記号-一般   
8    次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...             __記号-一般   
9    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...   tagger_名詞-固有名詞-一般   
10   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...     text_名詞-固有名詞-一般   
11   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...     text_名詞-固有名詞-一般   
12   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             、_記号-読点   
13   まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...             、_記号-読点   
14      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される             、_記号-読点   
15   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...             、_記号-読点   
16   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...         が_助詞-格助詞-一般   
17      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される         が_助詞-格助詞-一般   
18   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...            する_動詞-自立   
19   まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...            する_動詞-自立   
20      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される            する_動詞-自立   
21   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...            する_動詞-自立   
22   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...           て_助詞-接続助詞   
23   まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...           て_助詞-接続助詞   
24   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...         で_助詞-格助詞-一般   
25   まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...         で_助詞-格助詞-一般   
26   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...         で_助詞-格助詞-一般   
27   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...           など_助詞-副助詞   
28      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される           など_助詞-副助詞   
29   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...            の_助詞-連体化   
..                                                 ...                 ...   
75   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...          型_名詞-接尾-一般   
76   まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...      文字列_名詞-固有名詞-一般   
77   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...      文字列_名詞-固有名詞-一般   
78   まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...          群_名詞-接尾-一般   
79      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される        t_名詞-固有名詞-組織   
80   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...        t_名詞-固有名詞-組織   
81      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される           と_助詞-並立助詞   
82      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される           区切り_名詞-一般   
83      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される             間_名詞-一般   
84   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...            ')_記号-一般   
85   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...           ('＼_記号-一般   
86   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...         split_名詞-一般   
87   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...    split_名詞-固有名詞-組織   
88   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...       wakatid_名詞-一般   
89   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...             。_記号-句点   
90   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...            「＼_記号-一般   
91   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...           が_助詞-接続助詞   
92   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...              たい_助動詞   
93   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...      ため_名詞-非自立-副詞可能   
94   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...               だ_助動詞

In [9]:
idf_merge_df.isnull().any()

内容                         False
original_type              False
count                      False
word_count                 False
tf                         False
word_include_text_count    False
dtype: bool

In [10]:
# dfの分子（全文書数）を格納する新規カラムを追加
idf_merge_df['all_text_count'] = len(contents_merge_df['内容'].value_counts())
idf_merge_df

内容       original_type  \
0    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             (_記号-一般   
1    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             )_記号-一般   
2    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             ._記号-一般   
3    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...              1_名詞-数   
4    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...  NEologd_名詞-固有名詞-一般   
5    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...    Parse_名詞-固有名詞-一般   
6    まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...    Parse_名詞-固有名詞-一般   
7    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             __記号-一般   
8    次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...             __記号-一般   
9    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...   tagger_名詞-固有名詞-一般   
10   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...     text_名詞-固有名詞-一般   
11   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...     text_名詞-固有名詞-一般   
12   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             、_記号-読点   
13   まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...             、_記号-読点   
14      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される             、_記号-読点   
15   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...             、_記号-読点   
16   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...         が_助詞-格助詞-一般   
17      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される         が_助詞-格助詞-一般   
18   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...            する_動詞-自立   
19   まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...            する_動詞-自立   
20      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される            する_動詞-自立   
21   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...            する_動詞-自立   
22   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...           て_助詞-接続助詞   
23   まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...           て_助詞-接続助詞   
24   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...         で_助詞-格助詞-一般   
25   まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...         で_助詞-格助詞-一般   
26   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...         で_助詞-格助詞-一般   
27   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...           など_助詞-副助詞   
28      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される           など_助詞-副助詞   
29   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...            の_助詞-連体化   
..                                                 ...                 ...   
75   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...          型_名詞-接尾-一般   
76   まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...      文字列_名詞-固有名詞-一般   
77   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...      文字列_名詞-固有名詞-一般   
78   まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...          群_名詞-接尾-一般   
79      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される        t_名詞-固有名詞-組織   
80   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...        t_名詞-固有名詞-組織   
81      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される           と_助詞-並立助詞   
82      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される           区切り_名詞-一般   
83      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される             間_名詞-一般   
84   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...            ')_記号-一般   
85   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...           ('＼_記号-一般   
86   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...         split_名詞-一般   
87   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...    split_名詞-固有名詞-組織   
88   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...       wakatid_名詞-一般   
89   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...             。_記号-句点   
90   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...            「＼_記号-一般   
91   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...           が_助詞-接続助詞   
92   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...              たい_助動詞   
93   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...      ため_名詞-非自立-副詞可能   
94   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...               だ_助動詞

In [11]:
# idfを算出
# データフレーム型にapply関数を使う際には、axis=1を忘れないように注意
import math
idf_merge_df['idf'] = idf_merge_df.progress_apply(lambda x: math.log(x['all_text_count'] / x['word_include_text_count']) + 1 , axis=1)

In [12]:
idf_merge_df

内容       original_type  \
0    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             (_記号-一般   
1    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             )_記号-一般   
2    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             ._記号-一般   
3    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...              1_名詞-数   
4    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...  NEologd_名詞-固有名詞-一般   
5    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...    Parse_名詞-固有名詞-一般   
6    まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...    Parse_名詞-固有名詞-一般   
7    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             __記号-一般   
8    次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...             __記号-一般   
9    neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...   tagger_名詞-固有名詞-一般   
10   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...     text_名詞-固有名詞-一般   
11   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...     text_名詞-固有名詞-一般   
12   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             、_記号-読点   
13   まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...             、_記号-読点   
14      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される             、_記号-読点   
15   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...             、_記号-読点   
16   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...         が_助詞-格助詞-一般   
17      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される         が_助詞-格助詞-一般   
18   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...            する_動詞-自立   
19   まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...            する_動詞-自立   
20      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される            する_動詞-自立   
21   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...            する_動詞-自立   
22   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...           て_助詞-接続助詞   
23   まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...           て_助詞-接続助詞   
24   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...         で_助詞-格助詞-一般   
25   まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...         で_助詞-格助詞-一般   
26   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...         で_助詞-格助詞-一般   
27   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...           など_助詞-副助詞   
28      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される           など_助詞-副助詞   
29   neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...            の_助詞-連体化   
..                                                 ...                 ...   
75   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...          型_名詞-接尾-一般   
76   まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...      文字列_名詞-固有名詞-一般   
77   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...      文字列_名詞-固有名詞-一般   
78   まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...          群_名詞-接尾-一般   
79      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される        t_名詞-固有名詞-組織   
80   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...        t_名詞-固有名詞-組織   
81      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される           と_助詞-並立助詞   
82      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される           区切り_名詞-一般   
83      原形、品詞などの間には「＼t」が、分かち書きされた単語と単語の区切りには「＼n」が表示される             間_名詞-一般   
84   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...            ')_記号-一般   
85   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...           ('＼_記号-一般   
86   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...         split_名詞-一般   
87   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...    split_名詞-固有名詞-組織   
88   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...       wakatid_名詞-一般   
89   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...             。_記号-句点   
90   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...            「＼_記号-一般   
91   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...           が_助詞-接続助詞   
92   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...              たい_助動詞   
93   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...      ため_名詞-非自立-副詞可能   
94   次に邪魔な文字「＼t」を省きたい。よって、split('＼t')を使用したいがwakatid...               だ_助動詞

In [14]:
# csvファイルで出力
idf_merge_df.to_csv('idf_merge_df.csv', index=False, encoding='utf-8')

Mac環境の場合、csvファイルを出力した後

一旦CSVファイルを右クリックし、「このアプリケーションで開く」→「テキストエディット」を選択

command + shift + S をタップし、ファイルを複製

command + S をタップし保存画面を開く

「名前」を、任意の名前に「.csv」を付けた名前にして、「標準テキストのエンコーディング」を「日本語（Shift JIS）」として、保存

参考URL：https://support.freee.co.jp/hc/ja/articles/115002727126-CSV%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%81%8C%E6%96%87%E5%AD%97%E5%8C%96%E3%81%91%E3%81%97%E3%81%9F%E5%A0%B4%E5%90%88%E3%81%AE%E5%AF%BE%E5%87%A6%E6%96%B9%E6%B3%95#mac

In [15]:
# 上記作業終了後encoding='shift_jis'を設定してファイルを読み込む
idf_merge_df = pd.read_csv('idf_merge_df.csv', encoding='shift_jis')
idf_merge_df.head()

内容       original_type  \
0  neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             (_記号-一般   
1  neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             )_記号-一般   
2  neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...             ._記号-一般   
3  neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...              1_名詞-数   
4  neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...  NEologd_名詞-固有名詞-一般   

   count  word_count        tf  word_include_text_count  all_text_count  \
0      1          26  0.038462                        1               4   
1      1          26  0.038462                        1               4   
2      1          26  0.038462                        1               4   
3      1          26  0.038462                        1               4   
4      1          26  0.038462                        1               4   

        idf  
0  2.386294  
1  2.386294  
2  2.386294  
3  2.386294  
4  2.386294